In [1]:
import requests
from requests.auth import HTTPDigestAuth
import json
import time
import pandas as pd
import time
from pprint import pprint
from config import pet_finder_api_key, pet_finder_secret_key
petfinder_base_url = "https://api.petfinder.com/v2/"

ModuleNotFoundError: No module named 'config'

In [3]:
def getAccessToken():
    url = f'{petfinder_base_url}oauth2/token'
    #print(url)
    #format post request body data
    #https://www.geeksforgeeks.org/get-post-requests-using-python/
    # data to be sent to api; data is the POST body
    data = {'grant_type':"client_credentials", 
            'client_id':pet_finder_api_key, 
            'client_secret':pet_finder_secret_key
           }
    #print(data)
    #how to add headers in python POSTS
    #https://stackoverflow.com/questions/8685790/adding-header-to-python-requests-module
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    response = requests.post(url, data = data, headers=headers)
    #print(response)
    if(response.ok):
        jData = json.loads(response.content)
        #print(jData)
        return jData["access_token"]
    else:
        # If response code is not ok (200)
        return None


In [4]:
token = getAccessToken()

In [6]:
print(token)

None


In [20]:
def fetchPet(animal_type,breed,status):
    url = f'{petfinder_base_url}animals?type={animal_type}&breed={breed}&status={status}'
    print(url)
    #format for calls to animal api
    #https://api.petfinder.com/v2/animals?type=dog&page=2
    #headers: Authorization: Bearer eyJ0eXA...
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers )
    if(response.ok):
        jData = json.loads(response.content)
        return jData

    else:
        print("Not found")
        return None

In [21]:
fetchPet(animal_type="dog",breed="chihuahua",status="adoptable")

https://api.petfinder.com/v2/animals?type=dog&breed=chihuahua&status=adoptable
Not found


In [22]:
#function to get total pages of results
def totalPages(animal_type,status,limit):
    url = f'{petfinder_base_url}animals?type={animal_type}&status={status}&limit={limit}'
    print(url)
    #format for calls to animal api
    #https://api.petfinder.com/v2/animals?type=dog&page=2
    #headers: Authorization: Bearer eyJ0eXA...
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers )
    if(response.ok):
        jData = json.loads(response.content)
        return jData['pagination']['total_pages']
       
    else:
        print("Not found")
        return None



In [23]:
def fetchPet(animal_type,limit,status,page):
    url = f'{petfinder_base_url}animals?type={animal_type}&limit={limit}&status={status}&page={page}'
    print(url)
    #format for calls to animal api
    #https://api.petfinder.com/v2/animals?type=dog&page=2
    #headers: Authorization: Bearer eyJ0eXA...
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(url, headers=headers )
    if(response.ok):
        jData = json.loads(response.content)
        return jData

    else:
        print("Not found")
        return None    

In [24]:
total_pages = totalPages(animal_type="dog",status="adoptable", limit=100)
total_pages

https://api.petfinder.com/v2/animals?type=dog&status=adoptable&limit=100


1200

In [25]:
#change fetch_data to true if you want to fetch the data again
fetch_data = True

#to fetch data once and store in folder
if fetch_data == True:
    
    animal_type= "dog"
    limit = 100
    status = "adoptable"
    

    pets = []
    for  page in range(1,total_pages +1):
        petData = fetchPet(animal_type,limit,status,page)
        #print(petData)
        #append to a list from another list
        #https://stackabuse.com/append-vs-extend-in-python-lists/
        #pets.extend(petData["animals"])
        
        for row in petData["animals"]:
            
            pet_dict= {
            'pet_id': row['id'],
                'organization_id': row['organization_id'],
            'url': row['url'],
            'type': row['type'],
            'primary breed': row['breeds']['primary'],
            'secondary breed': row['breeds']['secondary'],
            'mixed breed': row['breeds']['mixed'],
            'age': row['age'],
            'gender': row['gender'],
            'size': row['size'],
            'photo1': '',
            'photo2': '',
            'photo3': '',
            'city': '',
            'state': '',
            'postcode': '',
            'email': '',
            'phone': ''
            }
            
            if len(row['photos']) > 0:
                pet_dict['photo1'] = row['photos'][0]['full']
            if len(row['photos']) > 1:
                pet_dict['photo2'] = row['photos'][1]['full']
            if len(row['photos']) > 2:
                pet_dict['photo3'] = row['photos'][2]['full']
                
                
                
            if 'contact' in row:
                contact = row['contact']
                
                if 'email' in contact:
                    pet_dict['email'] = row['contact']['email']
                if 'phone' in row:
                    pet_dict['phone'] = row['contact']['phone']
                
                if 'address' in contact:
                    address = row['contact']['address']
                    if 'city' in address:
                        pet_dict['city'] = row['contact']['address']['city']
                    if 'state' in address:
                        pet_dict['state'] = row['contact']['address']['state']
                    if 'postcode' in address:
                        pet_dict['postcode'] = row['contact']['address']['postcode']
        
                
            pets.append(pet_dict)    
            
            


https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=1
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=2
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=3
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=4
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=5
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=6
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=7
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=8
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=9
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=10
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=11
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=12
https://api.petfinder.com/v2/animals?

https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=103
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=104
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=105
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=106
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=107
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=108
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=109
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=110
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=111
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=112
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=113
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=114
https://api.petf

https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=203
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=204
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=205
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=206
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=207
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=208
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=209
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=210
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=211
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=212
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=213
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=214
https://api.petf

https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=303
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=304
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=305
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=306
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=307
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=308
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=309
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=310
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=311
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=312
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=313
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=314
https://api.petf

https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=403
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=404
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=405
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=406
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=407
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=408
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=409
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=410
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=411
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=412
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=413
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=414
https://api.petf

https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=503
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=504
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=505
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=506
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=507
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=508
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=509
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=510
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=511
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=512
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=513
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=514
https://api.petf

https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=603
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=604
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=605
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=606
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=607
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=608
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=609
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=610
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=611
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=612
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=613
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=614
https://api.petf

https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=703
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=704
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=705
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=706
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=707
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=708
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=709
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=710
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=711
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=712
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=713
https://api.petfinder.com/v2/animals?type=dog&limit=100&status=adoptable&page=714
https://api.petf

TypeError: 'NoneType' object is not subscriptable

In [28]:
pets_df = pd.DataFrame(pets)
pets_df

,age,city,email,gender,mixed breed,organization_id,pet_id,phone,photo1,photo2,photo3,postcode,primary breed,secondary breed,size,state,type,url
0,Young,Cleveland,citydogs@city.cleveland.oh.us,Male,True,OH1043,44293220,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,44113,Terrier,Pit Bull Terrier,Medium,OH,Dog,https://www.petfinder.com/dog/weezy-44293220/o...
1,Adult,Rose Hill,Kansasdiamonds@gmail.com,Female,True,KS305,44293226,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,,,67133,Shih Tzu,Miniature Poodle,Small,KS,Dog,https://www.petfinder.com/dog/suzie-44293226/k...
2,Young,Russellville,heavencanwaitanimalrescue@hotmail.com,Female,False,KY389,44293213,,,,,42276,Mixed Breed,None,Small,KY,Dog,https://www.petfinder.com/dog/allie-44293213/k...
3,Baby,Pompton Lakes,Doggiedogrescue@aol.com,Female,True,NJ519,44293221,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,,07442,Terrier,Dachshund,Small,NJ,Dog,https://www.petfinder.com/dog/lily-grace-44293...
4,Senior,Alvin,info@pughearts.com,Female,False,TX1084,44293212,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,,77512,Pug,None,Small,TX,Dog,https://www.petfinder.com/dog/lady-bug-4429321...
5,Adult,Pottsville,muttlover25@yahoo.com,Male,True,PA926,44293214,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,17901,Great Dane,Labrador Retriever,Large,PA,Dog,https://www.petfinder.com/dog/skittles-4429321...
6,Baby,Garden City,poetdogs@gmail.com,Female,True,MI935,44293206,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,48136,Golden Retriever,Collie,Medium,MI,Dog,https://www.petfinder.com/dog/beijing-44293206...
7,Senior,Scituate,None,Male,True,MA85,44293072,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,,,02066,Pug,Beagle,Medium,MA,Dog,https://www.petfinder.com/dog/yoda-44293072/ma...
8,Adult,Houston,info@redcollar.org,Female,True,TX1086,44293172,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,77002,Beagle,None,Medium,TX,Dog,https://www.petfinder.com/dog/prairie-rose-442...
9,Baby,West Richland,information@popptricities.org,Female,True,WA02,44293203,,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,99353,Dachshund,None,Small,WA,Dog,https://www.petfinder.com/dog/luna-44293203/wa...


In [29]:
pets_df.to_csv('Resources/pets.csv',header=True, index=False) 

In [30]:
pets_df.to_json("Resources/pets.json", orient="records")